In [34]:
import pandas as pd
import numpy as np


In [35]:
df = pd.read_csv('all_features.csv', index_col= None)
df = df.drop('Unnamed: 0', axis=1)
df.shape

(1524, 170)

In [36]:
# apply label encoder to the 'classified shape' column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['classified_shape'] = le.fit_transform(df['classified_shape'])
df.head()


,0,1,2,3,4,5,6,7,8,9,...,Width,Height,H_W_Ratio,Jaw_width,J_F_Ratio,MJ_width,MJ_J_width,filenum,filename,classified_shape
0,75.0,98.0,75.0,118.0,76.0,138.0,79.0,158.0,87.0,175.0,...,150.003333,212.084889,1.413868,69.065187,0.460424,122.036880,1.766981,0,Ciara (9).jpg,0
1,75.0,98.0,76.0,117.0,79.0,136.0,82.0,154.0,88.0,173.0,...,151.003311,210.038092,1.390950,79.000000,0.523167,125.000000,1.582278,1,Ciara (11).jpg,0
2,76.0,106.0,78.0,125.0,81.0,143.0,84.0,163.0,90.0,181.0,...,152.029602,218.036694,1.434173,70.007142,0.460484,124.004032,1.771305,2,Brittany Snow (7).jpg,0
3,71.0,99.0,74.0,119.0,77.0,139.0,80.0,159.0,88.0,177.0,...,157.012738,220.009091,1.401218,68.007353,0.433133,124.004032,1.823392,3,Julianne Hough (19).jpg,0
4,80.0,99.0,83.0,118.0,86.0,137.0,89.0,156.0,95.0,174.0,...,148.030402,214.233517,1.447226,70.007142,0.472924,121.000000,1.728395,4,Ashley Greene (24).jpg,0


In [37]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import ensemble
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.decomposition import PCA 

In [38]:
data = df.dropna(axis=0, how='any')
X = data
X = X.drop(['filenum','filename','classified_shape'] , axis = 1)
X_norm = normalize(X)
Y = data['classified_shape']

In [39]:
scaler = StandardScaler()  
scaler.fit(X)  

X = scaler.transform(X)

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X,Y,
    test_size=0.25,
    random_state=1200)

In [41]:
n_components = 18
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [42]:
# #Remove PCA 
X_train_pca = X_train
X_test_pca = X_test

In [45]:
# train a LDA classifier and print the accuracy and confusion matrix
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_pca, Y_train)
Y_pred = lda.predict(X_test_pca)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))


Accuracy: 0.6117021276595744
[[48  3 17  2  5]
 [ 7 31 19  0 11]
 [14  7 67  3  8]
 [ 4  3  7 41  7]
 [ 7  8  8  6 43]]
              precision    recall  f1-score   support

           0       0.60      0.64      0.62        75
           1       0.60      0.46      0.52        68
           2       0.57      0.68      0.62        99
           3       0.79      0.66      0.72        62
           4       0.58      0.60      0.59        72

    accuracy                           0.61       376
   macro avg       0.63      0.61      0.61       376
weighted avg       0.62      0.61      0.61       376



In [46]:
# train a MLP classifier and print the accuracy and confusion matrix
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
mlp.fit(X_train_pca, Y_train)
Y_pred = mlp.predict(X_test_pca)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

Accuracy: 0.6276595744680851
[[38  7 17  8  5]
 [ 9 39 11  0  9]
 [11  7 68  6  7]
 [ 4  2  9 42  5]
 [ 5  5  7  6 49]]
              precision    recall  f1-score   support

           0       0.57      0.51      0.54        75
           1       0.65      0.57      0.61        68
           2       0.61      0.69      0.64        99
           3       0.68      0.68      0.68        62
           4       0.65      0.68      0.67        72

    accuracy                           0.63       376
   macro avg       0.63      0.63      0.63       376
weighted avg       0.63      0.63      0.63       376



c:\Users\aksde\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [50]:
# save the lda model to disk 
import pickle
filename = 'lda_model.sav'
pickle.dump(lda, open(filename, 'wb'))

In [51]:
test_photo = 'data/pics/heart/amanda seyfried 10.jpg'
file_num = 2035

